# Trabajo Práctico Nro. 1
## Machine Learning 2.

Alumnos:
  - Crenna, Giuliano.
  - Sancho Almenar, Marinano.
  - Pace, Bruno.

## **DESCRIPCION**

En este problema, se presenta un conjunto de datos que contiene información sobre el rendimiento académico de estudiantes universitarios, así como diversos factores que podrían influir en él. El objetivo es construir un modelo de regresión utilizando redes neuronales para predecir el índice de rendimiento académico de los estudiantes basado en las características proporcionadas.

## **DATASET**

El dataset proporcionado incluye las siguientes variables para cada estudiante:
Hours Studied: El número total de horas dedicadas al estudio por cada estudiante.
Previous Scores: Las puntuaciones obtenidas por los estudiantes en exámenes previos.
Extracurricular Activities: Si el estudiante participa en actividades extracurriculares (Sí o No).
Sleep Hours: El número promedio de horas de sueño que el estudiante tuvo por día.
Sample Question Papers Practiced: El número de cuestionarios de muestra que el estudiante practicó.
Además, el dataset incluye la variable objetivo:
Performance Index: Un índice que representa el rendimiento académico general de cada estudiante, redondeado al entero más cercano. Este índice varía de 10 a 100, donde valores más altos indican un mejor rendimiento.

Utilizando el dataset proporcionado, el objetivo es construir un modelo de regresión utilizando redes neuronales que pueda predecir con precisión el índice de rendimiento académico de los estudiantes. Se debe entrenar y evaluar el modelo utilizando técnicas adecuadas de validación y métricas de evaluación de regresión.

## **OBJETIVO**

# Librerias

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# EDA

Descarga del dataset de forma automática.

In [2]:
!wget -O student.csv https://github.com/bpace1/ML-Redes-Densas-Convolucionales/raw/main/data/Student_Performance.csv

--2025-04-21 22:16:05--  https://github.com/bpace1/ML-Redes-Densas-Convolucionales/raw/main/data/Student_Performance.csv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bpace1/ML-Redes-Densas-Convolucionales/main/data/Student_Performance.csv [following]
--2025-04-21 22:16:05--  https://raw.githubusercontent.com/bpace1/ML-Redes-Densas-Convolucionales/main/data/Student_Performance.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175071 (171K) [text/plain]
Saving to: ‘student.csv’

student.csv         100%[===================>] 170.97K  --.-KB/s    in 0.004s  

2025-04-21 22:16:06 (38.1 MB/s

In [3]:
df = pd.read_csv('student.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  object 
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 468.9+ KB


# División Train y Validation.

Probar entrenando el modelo solo con las dos variables mas correlacionadas.

durante entrenamiento ver que metrica usamos. Func costo?

In [4]:
df_renamed = df.copy()
df_renamed.columns = df_renamed.columns.str.lower().str.replace(' ', '_')
df_renamed

,hours_studied,previous_scores,extracurricular_activities,sleep_hours,sample_question_papers_practiced,performance_index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0
...,...,...,...,...,...,...
9995,1,49,Yes,4,2,23.0
9996,7,64,Yes,8,5,58.0
9997,6,83,Yes,8,5,74.0
9998,9,97,Yes,7,0,95.0


In [5]:
df_train, df_test = train_test_split(df_renamed, test_size=.2, random_state=42)

# EDA

In [6]:
df_train_renamed = df_train.copy()
df_train_renamed.columns = df_train_renamed.columns.str.lower().str.replace(' ', '_')
df_train_renamed

,hours_studied,previous_scores,extracurricular_activities,sleep_hours,sample_question_papers_practiced,performance_index
9254,5,49,No,7,5,36.0
1561,2,48,Yes,7,6,25.0
1670,2,81,No,7,2,59.0
6087,2,46,No,6,1,22.0
6669,8,47,No,9,0,40.0
...,...,...,...,...,...,...
5734,8,50,Yes,6,6,48.0
5191,4,68,No,9,3,51.0
5390,9,48,No,7,6,44.0
860,1,47,No,9,0,20.0


In [7]:
df_train_encoded = df_train_renamed.copy()
df_train_encoded['extracurricular_activities'] = df_train_encoded['extracurricular_activities'].map({'Yes': 1, 'No': 0})

df_test_encoded = df_test.copy()
df_test_encoded ['extracurricular_activities'] = df_test_encoded['extracurricular_activities'].map({'Yes': 1, 'No': 0})

In [8]:
px.histogram(df_train_encoded)

In [9]:
px.scatter(df_train_encoded[['previous_scores', 'performance_index', 'hours_studied', 'sleep_hours','sample_question_papers_practiced']], title="Scatterplot de las variables")

In [10]:
px.box(df_train_encoded[['previous_scores', 'performance_index', 'hours_studied', 'sleep_hours', 'sample_question_papers_practiced']], title="Boxplot de las variables continuas")

In [11]:
px.scatter_matrix(df_train_encoded,
                  dimensions=['previous_scores', 'performance_index'], color='performance_index')

No se analizan outliers.

Estandarizacion de datos

In [12]:
df_train_normalized = df_train_encoded.copy()
df_test_normalized = df_test_encoded.copy()

scaler = MinMaxScaler()

numerical_features = ['hours_studied', 'previous_scores', 'sleep_hours', 'sample_question_papers_practiced', 'performance_index']

df_train_normalized[numerical_features] = scaler.fit_transform(df_train_normalized[numerical_features])
df_test_normalized[numerical_features] = scaler.transform(df_test_normalized[numerical_features])

In [13]:
df_for_corr = df_train_normalized.copy()
df_for_corr = df_for_corr.drop(columns=['extracurricular_activities'])

corr_matrix = df_for_corr.corr()
corr_matrix
fig = px.imshow(df_for_corr.corr(), zmin=-1, zmax=1, text_auto=True, title='Matriz de correlación de las variables', aspect='auto')
fig.show()


Podemos mencionar como destacable una única correlación fuerte entre previous-scores y performance_index. Esto indica que los alumnos tienden a mantener un rendimiento regular.

# Definición del modelo

In [14]:
X_train, y_train = df_train_normalized[['previous_scores','hours_studied']].to_numpy(), df_train_normalized['performance_index'].to_numpy()
X_test, y_test = df_test_normalized[['previous_scores','hours_studied']].to_numpy(), df_test_normalized['performance_index'].to_numpy()

In [15]:
X_train

array([[0.15254237, 0.5       ],
       [0.13559322, 0.125     ],
       [0.69491525, 0.125     ],
       ...,
       [0.13559322, 1.        ],
       [0.11864407, 0.        ],
       [0.10169492, 0.125     ]])

In [21]:
dim: int = X_train.shape[1]
print(dim)

loss = tf.keras.losses.MeanSquaredError()

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(dim,)),
    tf.keras.layers.Dense(units=32, activation='leaky_relu'),
    tf.keras.layers.Dense(units=32, activation='leaky_relu'),
    tf.keras.layers.Dense(units=32, activation='leaky_relu'),
    tf.keras.layers.Dense(units=16, activation='leaky_relu'),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test))

2
Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 3.8349e-04 - loss: 0.0238 - val_accuracy: 0.0000e+00 - val_loss: 0.0010
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 5.6070e-04 - loss: 7.5612e-04 - val_accuracy: 0.0000e+00 - val_loss: 0.0011
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 4.8064e-04 - loss: 7.9595e-04 - val_accuracy: 0.0000e+00 - val_loss: 7.4942e-04
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 5.1439e-04 - loss: 8.4405e-04 - val_accuracy: 0.0000e+00 - val_loss: 6.7529e-04
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 2.7854e-04 - loss: 7.0655e-04 - val_accuracy: 0.0000e+00 - val_loss: 7.0506e-04
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.5290e-04 - loss: 7.2887e-04 - val_accuracy: 0.0000e+00 - val_loss: 6.6317e-04
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 2.7890e-04 - loss: 7.0907e-04 - val_accuracy: 0.0000e+00 - val_loss: 7.0163e-04
Epoch 8/

KeyboardInterrupt: 

In [ ]:
X_train_comp = df_train_normalized.to_numpy()
X_test_comp = df_test_normalized.to_numpy()

In [ ]:
dim: int = X_train_comp.shape[1]
print(dim)

loss = tf.keras.losses.MeanSquaredError()

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(dim,)),
    tf.keras.layers.Dense(units=32, activation='leaky_relu'),
    tf.keras.layers.Dense(units=32, activation='leaky_relu'),
    tf.keras.layers.Dense(units=32, activation='leaky_relu'),
    tf.keras.layers.Dense(units=16, activation='leaky_relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


model.fit(X_train_comp, y_train, epochs=100, batch_size=32, validation_data=(X_test_comp, y_test))